In [ ]:
import numpy as np
from tvsclib.canonical_form import CanonicalForm
from tvsclib.mixed_system import MixedSystem
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.approximation import Approximation
import matplotlib.pyplot as plt
import tvsclib.utils as utils

# Notebook on approximations

This notebook is on approxiamtions of systems.
These are done by converting the system to a ordered representation.
When this is done, the singular values of the Hankel operators are stored.


## Introductary example
Lets create a matrix.
This has a two states and one state. We design the Hankel matrices in such a way that they have certian $\sigma$s

In [ ]:
#create a special matrix 
n = 10
mat = np.random.rand(2*n,2*n)
Uc, s, Vch = np.linalg.svd(mat[n:,:n], full_matrices=True)
Ua, s, Vah = np.linalg.svd(mat[:n,n:], full_matrices=True)

sc = np.linspace(5,0,n)
sa = np.linspace(5,0,n)

#sc = np.zeros(n)
#sa = np.zeros(n)
#sc[0]=5
#sc[1]=5
#sa[0]=5
#sa[1]=0

mat[n:,:n] = Uc*sc@Vch
mat[:n,n:] = Ua*sa@Vah

T = ToeplitzOperator(mat, [n,n], [n,n])
S = SystemIdentificationSVD(T,epsilon=1e-10)
system = MixedSystem(S)

utils.show_system(system)
plt.clim(-1.5,1.5)

Lets have a look at the sigmas of the system and check if they are tha same as the designated sigmas

In [ ]:
approx =Approximation(system)
display(approx.sigmas_causal)
display(sc)


display(approx.sigmas_anticausal)
display(sa)

Now create a approxiamtion:

In [ ]:
eps = 3

approx_system=approx.get_approxiamtion(eps)
utils.show_system(approx_system)
plt.clim(-1.5,1.5)

We can now see that the number of states is reduced. we have as many states as we have $\sigma_i > \epsilon$

In [ ]:
print(approx_system)

In [ ]:
#check the system by creating a reference matrix
mat_ref = mat.copy()
nc = np.count_nonzero(sc>eps)
na = np.count_nonzero(sa>eps)

mat_ref[n:,:n] = Uc[:,:nc]*sc[:nc]@Vch[:nc,:]
mat_ref[:n,n:] = Ua[:,:na]*sa[:na]@Vah[:na,:]

dif = mat_ref-approx_system.to_matrix()
plt.matshow(dif)
plt.clim(-1.5e-14,1.5e-14)
print(np.max(dif))

## More complex system
Now we introduce the Hankel norm

$$\|T\|_H = \sup_{i}\|T_i\|$$

This is the supremum over the spectral norm (the matrix 2-norm) of each individual Hankel matrix.

The Hankel norm approxiamtion satisfies the condition 

$$\|T-\tilde{T}\|_H \leq \gamma $$

See also notes down below


For details see: page 9f and 264ff of
P. Dewilde und A.-J. van der Veen, Time-Varying Systems and Computations. Boston, MA: Springer US, 1998. doi: 10.1007/978-1-4757-2817-0.


In [ ]:
#Hankel norm etc....
def hankelnorm(A, dims_in,dims_out):
    #for more details on the implementation see the notebook on Obs and Reach
    n = len(dims_in)
    s_c = [np.max(np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)) for k in range(1,n)]
    s_a = [np.max(np.linalg.svd(A[:np.sum(dims_out[:k+1]),-np.sum(dims_in[k+1:]):],compute_uv=False)) for k in range(n-2,-1,-1)]
    return max(max(s_c),max(s_a))

In [ ]:
dims_in =  [10]*8
dims_out = [10]*8
matrix = np.random.rand(sum(dims_out), sum(dims_in))
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-10)
system = MixedSystem(S)

print(system)

In [ ]:
eps = 1
approx =Approximation(system)
system_approx = approx.get_approxiamtion(eps)
print(system_approx)

Now we compute the Hankelnorm of the matrix and compare it to the $\sigma$s obtained by the approxiamtion algorithm. We can see that biggest $\sigma$ is equal to the hankel norm

In [ ]:
hankelnorm(matrix,dims_in,dims_out)

In [ ]:
[np.max(approx.sigmas_anticausal[i]) for i in range(len(approx.sigmas_anticausal))]

In [ ]:
[np.max(approx.sigmas_causal[i]) for i in range(len(approx.sigmas_anticausal))]

Now we Obtain the hankel norm of the difference. We will see that $\|T-\tilde{T}\|_H$ is often slighly bigger than $\epsilon$. 

In [ ]:
hankelnorm(system_approx.to_matrix()-system.to_matrix(),dims_in,dims_out)

In [ ]:
#get system using identifucation
S_appr = SystemIdentificationSVD(T,epsilon=eps,relative=False)
system_approx_ident = MixedSystem(S_appr)
print(system_approx_ident)

In [ ]:
np.max(abs(system_approx_ident.to_matrix()-system_approx.to_matrix()))

In [ ]:
hankelnorm(system_approx_ident.to_matrix()-system.to_matrix(),dims_in,dims_out)

Note here: It is also possible to obtain more genreal Hankel norm approxiamtions. These sattsify the condition:

$$\| \Gamma^{-1}(T-\tilde{T})\|_H \leq 1 $$

Where $\Gamma$ is a diagonal and hermition operator. If we set $\Gamma = I\gamma$ we obtian the condition described above